# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import os
import re
import pickle
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zeyne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zeyne\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zeyne\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponsetable', con=engine.connect())

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    cleaned_tokens = []
    for tok in tokens:
        cleaned_tok = lemmatizer.lemmatize(tok).lower().strip()
        cleaned_tokens.append(cleaned_tok)
    
    return cleaned_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
            
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)

C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000027CFB88DC10>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
category_names = list(y.columns) 
for i in range(len(category_names)):
    print('=========================================================')
    print("Category:",category_names[i])
    print('=========================================================')
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i], zero_division=0))

Category: related
              precision    recall  f1-score   support

           0       0.74      0.26      0.38      1868
           1       0.81      0.97      0.88      5941

    accuracy                           0.80      7809
   macro avg       0.78      0.62      0.63      7809
weighted avg       0.79      0.80      0.76      7809

Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      6476
           1       0.88      0.43      0.58      1333

    accuracy                           0.89      7809
   macro avg       0.89      0.71      0.76      7809
weighted avg       0.89      0.89      0.88      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# get list of all model parameters to select some 
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__bootstrap',
 'clf__estimator__ccp_alpha',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__max_samples',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',
 'clf__n_jobs',
 'features',
 'features__n_jobs',
 'features__text_pipeline',
 'features__text_pipeline__memory',
 'features__text_pipeline__steps',
 'features__text_pipeline__tfidf',
 'features__text_pipeline__tfidf__norm',
 'features__text_pipeline__tfidf__smooth_idf',
 'features__text_pipeline__tfidf__sublinear_tf',
 'features__text_pipeline__tfidf__use_idf',
 'features_

In [11]:
#select only 2 parameters to test the method due to the grid search is time intensive
parameters = {
        'clf__estimator__max_features': ['sqrt', 'log2'],
        'clf__estimator__max_samples': [1,5],
    
    }

cv = GridSearchCV(pipeline, param_grid=parameters) 

cv.fit(X_train, y_train)

C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: User

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000027CFB88DC10>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_features': ['sqrt', 'log2'],
                         'clf__estimator__max_samples': [1, 5]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred = pipeline.predict(X_test)
for i in range(len(category_names)):
    print('=========================================================')
    print("Category:",category_names[i])
    print('=========================================================')
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i], zero_division=0))

Category: related
              precision    recall  f1-score   support

           0       0.74      0.26      0.38      1868
           1       0.81      0.97      0.88      5941

    accuracy                           0.80      7809
   macro avg       0.78      0.62      0.63      7809
weighted avg       0.79      0.80      0.76      7809

Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      6476
           1       0.88      0.43      0.58      1333

    accuracy                           0.89      7809
   macro avg       0.89      0.71      0.76      7809
weighted avg       0.89      0.89      0.88      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [14]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [15]:
pipeline2.fit(X_train, y_train)

C:\Users\zeyne\JJ\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000027CFB88DC10>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [16]:
y_pred = pipeline.predict(X_test)

for i in range(len(category_names)):
    print('=========================================================')
    print("Category:",category_names[i])
    print('=========================================================')
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i], zero_division=0))

Category: related
              precision    recall  f1-score   support

           0       0.74      0.26      0.38      1868
           1       0.81      0.97      0.88      5941

    accuracy                           0.80      7809
   macro avg       0.78      0.62      0.63      7809
weighted avg       0.79      0.80      0.76      7809

Category: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      6476
           1       0.88      0.43      0.58      1333

    accuracy                           0.89      7809
   macro avg       0.89      0.71      0.76      7809
weighted avg       0.89      0.89      0.88      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

### 9. Export your model as a pickle file

In [17]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(pipeline2, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.